In [12]:
import numpy as np
import os
#import matplotlib.pyplot as plt

from azureml.core.dataset import Dataset
from azureml.core.datastore import Datastore
import pandas as pd

import diff_classifier

import mrmr


In [13]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.49.0


### If running in the East US region, run the cell below

In [14]:
ws = Workspace.get(
    name='nels_east_us',
    subscription_id='9c49afdd-cccb-44e2-a4eb-5b2f941c774c',
    resource_group='schimek_cloud',
    location='eastus')

In [5]:
ws.name

'schimek_mpt_eastus2'

### if running in East US 2 region, run the cell below

In [3]:
ws = Workspace.get(
    name='schimek_mpt_eastus2',
    subscription_id='9c49afdd-cccb-44e2-a4eb-5b2f941c774c',
    resource_group='schimek_cloud',
    location='eastus2')

### If running in West US 2, run the cell below

In [14]:
ws = Workspace.get(
    name='schimek_west_us_2',
    subscription_id='9c49afdd-cccb-44e2-a4eb-5b2f941c774c',
    resource_group='schimek_cloud',
    location='westus2')

### If runnign west US 3, run the cell below

In [24]:
ws = Workspace.get(
    name='schimek_west_us_3',
    subscription_id='9c49afdd-cccb-44e2-a4eb-5b2f941c774c',
    resource_group='schimek_cloud',
    location='westus3')

In [16]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
if ws.name == 'nels_east_us':
    cluster_name = 'nlsschim1'
elif ws.name == 'schimek_mpt_eastus2':
    cluster_name = "nlsschim-gpu-cluster"
elif ws.name == 'schimek_west_us_2':
    cluster_name = "schimek-cluster-gpu"
elif ws.name == 'schimek_west_us_3':
    cluster_name = "schimek-cpu"
print(cluster_name)

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=1,
                                                           min_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())


nlsschim1
Found existing compute target

Running
{'errors': [], 'creationTime': '2023-08-07T18:00:36.723624+00:00', 'createdBy': {'userObjectId': '06e4aaf0-2fa3-4116-b650-3be63aa1a058', 'userTenantId': 'f6b6dd5b-f02f-441a-99a0-162ac5060bd2', 'userName': 'Nels Schimek'}, 'modifiedTime': '2023-09-19T23:15:11.116786+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS12_V2'}


In [17]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

schimek-gpu-cluster AmlCompute Failed
nlsschim1 ComputeInstance Succeeded


In [23]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.7
- pip=21.3.1
- py-xgboost#-gpu
- pip:
  - h5py<=2.10.0
  - azureml-defaults
  - keras<=2.3.1
  - matplotlib
  - protobuf==3.20.1
  - stochastic
  - diff_classifier
  - scikit-learn
  - polars
  - mrmr-selection
  - mlflow
  - azureml-mlflow

Overwriting conda_dependencies.yml


In [24]:
from azureml.core import Environment

keras_env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = './conda_dependencies.yml')

# Specify a GPU base image
##keras_env.docker.enabled = True
#keras_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu20.04'

In [ ]:
dataset = Dataset.get_by_name(ws, name='age_mpt_features')
dataset.to_path()

In [31]:
datastore = Datastore.get(ws, 'workspaceblobstore')
file_dataset = Dataset.File.from_files(path=(datastore, 'UI/2023-09-06_223916_UTC/'))



In [32]:
# Confirm the correct dataset is being used
file_dataset.to_path()

['/features_P14_40nm_s1_v1.csv',
 '/features_P14_40nm_s1_v2.csv',
 '/features_P14_40nm_s1_v3.csv',
 '/features_P14_40nm_s1_v4.csv',
 '/features_P14_40nm_s1_v5.csv',
 '/features_P14_40nm_s2_v1.csv',
 '/features_P14_40nm_s2_v2.csv',
 '/features_P14_40nm_s2_v3.csv',
 '/features_P14_40nm_s2_v4.csv',
 '/features_P14_40nm_s2_v5.csv',
 '/features_P14_40nm_s3_v1.csv',
 '/features_P14_40nm_s3_v2.csv',
 '/features_P14_40nm_s3_v3.csv',
 '/features_P14_40nm_s3_v4.csv',
 '/features_P14_40nm_s3_v5.csv',
 '/features_P21_40nm_s1_v1.csv',
 '/features_P21_40nm_s1_v2.csv',
 '/features_P21_40nm_s1_v3.csv',
 '/features_P21_40nm_s1_v4.csv',
 '/features_P21_40nm_s1_v5.csv',
 '/features_P21_40nm_s2_v1.csv',
 '/features_P21_40nm_s2_v2.csv',
 '/features_P21_40nm_s2_v3.csv',
 '/features_P21_40nm_s2_v4.csv',
 '/features_P21_40nm_s2_v5.csv',
 '/features_P21_40nm_s3_v1.csv',
 '/features_P21_40nm_s3_v2.csv',
 '/features_P21_40nm_s3_v3.csv',
 '/features_P21_40nm_s3_v4.csv',
 '/features_P21_40nm_s3_v5.csv',
 '/feature

In [33]:
dataset = file_dataset.register(workspace = ws, name = 'age_all_ages_features', create_new_version=True)

In [25]:
from azureml.core import ScriptRunConfig

#args = ['--input-data', dataset.as_named_input("trajectories")]#,
#         '--batch-size', 50,
#         '--first-layer-neurons', 300,
#         '--second-layer-neurons', 100,

src = ScriptRunConfig(source_directory='.',
                      script='azure_datapoint_eval_script.py',
                      #arguments=args,
                      compute_target=compute_target,
                      environment=keras_env)

In [26]:
from azureml.core import Experiment
exp = Experiment(workspace=ws, name='azure_OGD_P17_STR_3div_10_percent_data')

In [27]:
run = exp.submit(src)

In [41]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…